### Imports and utils

Ensure that OpenSlide is installed. If you are using the provided environment, it should already be installed. To set up the environment and download the necessary packages, please follow the steps outlined in the README and install.txt files.

In [1]:
!pip install openslide-python

  Using cached openslide_python-1.3.1-cp39-cp39-linux_x86_64.whl

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Import necessary packages

In [4]:
import openslide 
import numpy as np 
import pandas as pd 
import cv2 
from scipy.stats import scoreatpercentile
from PIL import Image
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from tqdm import tqdm 
import time
from sklearn.model_selection import train_test_split
import csv
import zipfile 
import tempfile 
from skimage import io, color
from skimage.exposure import is_low_contrast

Adjust the folder_path as needed to match your directory. The images should be placed in the specified folder.

In [3]:
folder_path = './data/images'
level = 2
patch_size=224
step_size=200
patch_file_name='more_selective_patches_7'
patches_path=f'./patches_csv/{patch_file_name}_level_{level}_size_{patch_size}_step_{step_size}.csv'
csv_dir = f'./patches_csv/{patch_file_name}_level_{level}_size_{patch_size}_step_{step_size}'
csv_file=f'./patches_csv/{patch_file_name}_level_{level}_size_{patch_size}_step_{step_size}.csv'
zip_name = f"{patch_file_name}_level_{level}_size_{patch_size}_step_{step_size}.zip"

# Ensure the directory patches_csv and data exists as well as the image folder.
os.makedirs(os.path.dirname(patches_path), exist_ok=True)
os.makedirs(folder_path, exist_ok=True)

## Preprocessing Functions
The following functions are provided for preprocessing whole slide images.

In [4]:
def scaling_images(slide, x, y, width, height, level):
    """
    Scales the coordinates and dimensions of a patch to the base level (level 0) of a whole slide image.

    Parameters:
    slide (openslide.OpenSlide): An OpenSlide object representing the whole slide image.
    x (int): The x-coordinate of the top-left corner of the patch at the specified level.
    y (int): The y-coordinate of the top-left corner of the patch at the specified level.
    width (int): The width of the patch at the specified level.
    height (int): The height of the patch at the specified level.
    level (int): The magnification level (0-5, where 0 is the base level and 5 is the most zoomed-out).

    Returns:
    tuple: Scaled coordinates and dimensions (base_x, base_y, base_width, base_height) at the base level.
    """
 
    scale_factor = slide.level_downsamples[level]
    base_x = int(x * scale_factor)
    base_y = int(y * scale_factor)
    base_width = int(width * scale_factor)
    base_height = int(height * scale_factor)
    return base_x, base_y, base_width, base_height


In [5]:
def check_patch_not_within_bounds(slide, base_width, base_height, base_x, base_y):
    """
    Checks if the requested patch is within the bounds of the base level (level 0) of a whole slide image.

    Parameters:
    slide (openslide.OpenSlide): An OpenSlide object representing the whole slide image.
    base_height (int): The height of the patch at the base level.
    base_width (int): The width of the patch at the base level.
    base_x (int): The x-coordinate of the top-left corner of the patch at the base level.
    base_y (int): The y-coordinate of the top-left corner of the patch at the base level.

    Returns:
    int: Returns 1 if the patch is out of bounds, otherwise returns 0.
    """
    base_dims = slide.level_dimensions[0]
    if base_x + base_width > base_dims[0] or base_y + base_height > base_dims[1]:
        print('x', base_x + base_width, base_dims[0])
        print('y', base_y + base_height, base_dims[1])
        print("Requested patch is out of bounds.")
        return 1
    return 0


In [6]:
def show_image_patch(slide, x, y, width, height, level):
    """
    Displays a specified patch from a pathology image file at a given magnification level.

    Parameters:
    tif_file (str): The path to the TIFF image file.
    x (int): The x-coordinate of the top-left corner of the patch.
    y (int): The y-coordinate of the top-left corner of the patch.
    width (int): The width of the patch.
    height (int): The height of the patch.
    level (int): The magnification level (0-5, where 5 is the most zoomed-out).

    Returns:
    None
    """

    # Scale the coordinates and dimensions to the base level (level 0)
    base_x, base_y, base_width, base_height = scaling_images(slide, x, y, width, height, level)

    #Check if patch is within bounds
    if check_patch_not_within_bounds(slide,base_width, base_height, base_x, base_y):
        return
    
    # Read a region of the slide (patch) at the base level
    patch = slide.read_region((base_x, base_y), level, (width, height))

    return patch


In [7]:
def resize_bboxes(bboxes, old_size, new_size):
    """
    Scale the bounding boxes accordingly.
    
    :param bboxes: List of bounding boxes, each in the format [x_min, x_max, y_min, y_max].
    :param display_size: Tuple (new_width, new_height) indicating the display size for the image.
    :param size_0: Tuple (width, height) indicating the size 0 for the image.
    :return: Resized image and scaled bounding boxes.
    """
    
    # Scale bounding boxes
    x_scale = new_size[0] / old_size[0]
    y_scale = new_size[1] / old_size[1]
    
    scaled_bboxes = []
    for bbox in bboxes:
        x_min, x_max, y_min, y_max = bbox
        new_x_min = int(x_min * x_scale)
        new_y_min = int(y_min * y_scale)
        new_x_max = int(x_max * x_scale)
        new_y_max = int(y_max * y_scale)
        scaled_bboxes.append([new_x_min, new_x_max, new_y_min, new_y_max])
    
    return scaled_bboxes


In [14]:
def get_bounding_boxes_for_specific_image(image_name, split='train'):
    """
    prints out the bounding boxes for a specific image file
    """
    #Read in the training data
    # Find the row with the specified filename
    #split = is_image_train_val_annotated(image_name)
    #if split is None:
    #    return
    data = pd.read_csv(f'./data/{split}.csv')
    row = data[data['filename'] == image_name]
    row = row.reset_index()
    size = (row.loc[0, "max_x"], row.loc[0, "max_y"])
    bboxes = list(row[["x1", "x2", "y1", "y2"]].itertuples(index=False, name=None))
    return bboxes,  size[0], size[1]


In [8]:
def is_giou_greater(box1, box2):
    """
    Check if two bounding boxes intersect and if the GIoU between them is at least 0.5.
    
    Parameters:
    - box1, box2: Lists or tuples of format [x1_min, y1_min, x1_max, y1_max] where
      (x1_min, y1_min) is the top-left coordinate and (x1_max, y1_max) is the bottom-right coordinate.
    
    Returns:
    - True if the GIoU is at least 0.5, False otherwise.
    """
    
    # Unpack the coordinates of both bounding boxes
    x1_min, x1_max, y1_min, y1_max = box1
    x2_min, x2_max, y2_min, y2_max = box2
    
    # Calculate the area of both bounding boxes
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    
    # Calculate the intersection coordinates
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    
    # Check if there is an intersection
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return False
    
    # Calculate the area of the intersection
    inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    # Calculate the union area
    union_area = area1 + area2 - inter_area
    # Calculate IoU
    iou = inter_area / union_area
    
    # Calculate the coordinates of the smallest enclosing box
    enc_x_min = min(x1_min, x2_min)
    enc_y_min = min(y1_min, y2_min)
    enc_x_max = max(x1_max, x2_max)
    enc_y_max = max(y1_max, y2_max)
    
    # Calculate the area of the smallest enclosing box
    enc_area = (enc_x_max - enc_x_min) * (enc_y_max - enc_y_min)
    
    # Calculate GIoU
    giou = iou - (enc_area - union_area) / enc_area
    
    # Return True if GIoU is at least 0.5, otherwise False
    return giou #>= 0.2



In [10]:
### Annotation and label utils

def is_image_train_val_annotated(image_name):
    data = pd.read_csv('clean_train.csv')
    rows = data[data['filename'] == image_name]
    if len(rows)>0:
        return 'train'
    data = pd.read_csv('./data/validation.csv')
    rows = data[data['filename'] == image_name]
    if len(rows)>0:
        return 'validation'
    return 


def is_patch_positive(patch_bbox, ground_truth_bboxes):
    max_giou = 0.0
    for bbox in ground_truth_bboxes:
        new_giou = is_giou_greater(patch_bbox, bbox)
        if new_giou > max_giou:
            max_giou = new_giou
    return max_giou


In [11]:
def count_the_number_of_total_patches(image_path, level, patch_size=200, step_size=512):
    slide = openslide.OpenSlide(image_path)
    width, height = slide.level_dimensions[level]
    count = 0
    total_patches_x = (width - patch_size) // step_size + 1
    total_patches_y = (height - patch_size) // step_size + 1
    total_patches = total_patches_x * total_patches_y
    return width,  height

def get_width_height(image_path, level):
    slide = openslide.OpenSlide(image_path)
    width, height = slide.level_dimensions[level]
    return width,  height


In [12]:
def is_background_patch_ambrosini(patch, od_threshold=0.12):
    od = optical_density_transformation_ambrosini(patch)
    # Check if any channel has OD less than the threshold
    is_background = np.any(od < od_threshold, axis=2)
    # Calculate the ratio of background pixels
    background_ratio = np.mean(is_background)
    return background_ratio > 0.995  # More than 99.5% of the patch is background
    
def optical_density_transformation_ambrosini(patch):
    # Convert patch to numpy array
    patch_array = np.array(patch) / 255.0
    # Calculate optical density
    od = -np.log10(patch_array + 1e-6)
    return od

def is_patch_white(patch, threshold_white=0.95, brightness_threshold=200):
    # Convert patch to grayscale
    gray_patch = patch.convert('L')
    # Convert to numpy array
    gray_array = np.array(gray_patch)
    # Calculate the number of white pixels
    white_pixels = np.sum(gray_array >= brightness_threshold)
    total_pixels = gray_array.size
    # Calculate the proportion of white pixels
    white_ratio = white_pixels / total_pixels
    # Check if the patch is more than threshold_white% white
    return white_ratio >= threshold_white


In [13]:
def hsv_color_mask(patch):
    hsv_patch = cv2.cvtColor(np.array(patch), cv2.COLOR_RGB2HSV)

    # Define HSV ranges for blue, purple, and pink colors
    lower_blue = np.array([100, 50, 50])
    upper_blue = np.array([140, 255, 255])
    
    lower_purple = np.array([130, 50, 50])
    upper_purple = np.array([160, 255, 255])
    
    lower_pink = np.array([160, 50, 50])
    upper_pink = np.array([180, 255, 255])

    # Create masks for each color
    mask_blue = cv2.inRange(hsv_patch, lower_blue, upper_blue)
    mask_purple = cv2.inRange(hsv_patch, lower_purple, upper_purple)
    mask_pink = cv2.inRange(hsv_patch, lower_pink, upper_pink)
    
    # Combine the masks
    combined_mask = cv2.bitwise_or(mask_blue, mask_purple)
    combined_mask = cv2.bitwise_or(combined_mask, mask_pink)

    return combined_mask

def calculate_color_coverage(mask):
    return np.sum(mask > 0) / mask.size

def is_relevant_patch_color(patch, color_threshold=0.05):
    mask = hsv_color_mask(patch)
    color_coverage = calculate_color_coverage(mask)
    return color_coverage >= color_threshold


### Staining normalization

In [14]:

"""
Mitkovetta. (n.d.). normalizeStaining.m. GitHub repository.
Retrieved from https://github.com/mitkovetta/staining-normalization/blob/master/normalizeStaining.m
[1] A method for normalizing histology slides for quantitative analysis, M Macenko, M Niethammer, JS Marron, D Borland, JT Woosley, G Xiaojun, C Schmitt, NE Thomas, IEEE ISBI, 2009. dx.doi.org/10.1109/ISBI.2009.5193250
This MATLAB code was translated to Python with assistance from ChatGPT, developed by OpenAI.
"""
def normalize_staining(I, Io=240, beta=0.15, alpha=1, HERef=None, maxCRef=None):
    """
    Normalize the staining appearance of images originating from H&E stained sections.

    Parameters:
        I (numpy.ndarray): RGB input image.
        Io (int, optional): Transmitted light intensity. Default is 240.
        beta (float, optional): OD threshold for transparent pixels. Default is 0.15.
        alpha (float, optional): Tolerance for the pseudo-min and pseudo-max. Default is 1.
        HERef (numpy.ndarray, optional): Reference H&E OD matrix. Default value is defined.
        maxCRef (numpy.ndarray, optional): Reference maximum stain concentrations for H&E. Default value is defined.

    Returns:
        Inorm (PIL.Image.Image): Normalized image.
        H (PIL.Image.Image, optional): Hematoxylin image.
        E (PIL.Image.Image, optional): Eosin image.
    """
    # Default reference H&E OD matrix
    if HERef is None:
        HERef = np.array([
            [0.5626, 0.2159],
            [0.7201, 0.8012],
            [0.4062, 0.5581]
        ])
    # Default reference maximum stain concentrations
    if maxCRef is None:
        maxCRef = np.array([1.9705, 1.0308])
    # Convert PIL image to numpy array if necessary
    if isinstance(I, Image.Image):
        I = np.array(I)
    
    h, w, c = I.shape
    I = I.reshape((-1, 3)).astype(np.float64)
    
    # calculate optical density
    OD = -np.log((I + 1) / Io)
    
    # remove transparent pixels
    ODhat = OD[np.all(OD >= beta, axis=1)]
    
    # calculate eigenvectors
    _, V = np.linalg.eigh(np.cov(ODhat.T))
    
    # project on the plane spanned by the eigenvectors corresponding to the two largest eigenvalues
    That = np.dot(ODhat, V[:, 1:3])
    
    # find the min and max vectors and project back to OD space
    phi = np.arctan2(That[:, 1], That[:, 0])
    
    minPhi = scoreatpercentile(phi, alpha)
    maxPhi = scoreatpercentile(phi, 100 - alpha)
    
    vMin = np.dot(V[:, 1:3], [np.cos(minPhi), np.sin(minPhi)])
    vMax = np.dot(V[:, 1:3], [np.cos(maxPhi), np.sin(maxPhi)])
    # Heuristic to make the vector corresponding to hematoxylin first and eosin second
    if vMin[0] > vMax[0]:
        HE = np.column_stack((vMin, vMax))
    else:
        HE = np.column_stack((vMax, vMin))
    
    Y = OD.T
    # Determine concentrations of the individual stains
    C = np.linalg.lstsq(HE, Y, rcond=None)[0]
    # Normalize stain concentrations
    maxC = np.percentile(C, 99, axis=1)
    
    C = C / maxC[:, np.newaxis]
    C = C * maxCRef[:, np.newaxis]
    # Recreate the image using reference mixing matrix
    Inorm = Io * np.exp(-np.dot(HERef, C))
    Inorm = Inorm.T.reshape(h, w, 3)
    Inorm = np.clip(Inorm, 0, 255).astype(np.uint8)
    Inorm = Image.fromarray(Inorm)
    
    H, E = None, None
    
    if C.shape[0] > 1:
        H = Io * np.exp(-np.outer(HERef[:, 0], C[0, :]))
        H = H.T.reshape(h, w, 3)
        H = np.clip(H, 0, 255).astype(np.uint8)
        H = Image.fromarray(H)
    
    if C.shape[0] > 1:
        E = Io * np.exp(-np.outer(HERef[:, 1], C[1, :]))
        E = E.T.reshape(h, w, 3)
        E = np.clip(E, 0, 255).astype(np.uint8)
        E = Image.fromarray(E)
    
    return Inorm, H, E

"""
## How to call the function:
Inorm, H, E = normalize_staining(patch)


###
Definitions:
	•	Normalized Image (Inorm): Appears similar to the original RGB image but with adjusted colors and intensity
        to match a standardized reference. It includes contributions from both Hematoxylin (blue/purple)
        and Eosin (pink/red) stains.
	•	Hematoxylin Image (H): Primarily shows the nuclei stained by Hematoxylin in shades of blue or purple.
        It lacks the pink/red hues contributed by Eosin.
    •	Eosin Image (E): The Eosin image is a single-channel image that represents only the Eosin stain component of the original image. Eosin typically stains the cytoplasm and extracellular matrix in various shades of pink or red.

"""




'\n## How to call the function:\nInorm, H, E = normalize_staining(patch)\n\n\n###\nDefinitions:\n\t•\tNormalized Image (Inorm): Appears similar to the original RGB image but with adjusted colors and intensity\n        to match a standardized reference. It includes contributions from both Hematoxylin (blue/purple)\n        and Eosin (pink/red) stains.\n\t•\tHematoxylin Image (H): Primarily shows the nuclei stained by Hematoxylin in shades of blue or purple.\n        It lacks the pink/red hues contributed by Eosin.\n    •\tEosin Image (E): The Eosin image is a single-channel image that represents only the Eosin stain component of the original image. Eosin typically stains the cytoplasm and extracellular matrix in various shades of pink or red.\n\n    -> we are interested in either Inorm or H. Eosin is just the pink stuff without the \'cells\'.\n    I would suggest trying out the \'H\' because it shows the cells which we are interested in.\n    \n\n\n##How to display the original and norm

In [15]:

def remove_colors(image, lower_color_bounds, upper_color_bounds):
    """
    Remove specified color ranges from the image.

    Parameters:
    image (numpy array): The input image in OpenCV format.
    lower_color_bounds (list of numpy arrays): The lower bounds for colors to be removed.
    upper_color_bounds (list of numpy arrays): The upper bounds for colors to be removed.

    Returns:
    numpy array: The image with specified colors removed.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(image.shape[:2], dtype="uint8")

    for lower, upper in zip(lower_color_bounds, upper_color_bounds):
        color_mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.bitwise_or(mask, color_mask)

    result = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))
    return result, mask

def highlight_artifacts(original_image, processed_image):
    """
    Highlight artifacts by subtracting processed image from the original image.

    Parameters:
    original_image (numpy array): The original image.
    processed_image (numpy array): The processed image with colors removed.

    Returns:
    numpy array: The image highlighting the artifacts.
    """
    difference = cv2.absdiff(original_image, processed_image)
    _, highlighted = cv2.threshold(difference, 30, 255, cv2.THRESH_BINARY)
    return highlighted

def remove_artifacts(patch):
    """
    Process a pathology image patch to remove artifacts by:
    1. Detecting regions of interest (ROIs) based on color.
    2. Highlighting artifacts.
    3. Keeping only the ROIs in the original image while making the rest white.

    Example Usage:
        image_id = '2qj5MlLLBT_a.tif'
        image_path = os.path.join('data/images', image_id)
        slide = openslide.OpenSlide(image_path)
        patch = show_image_patch(slide, 2000, 63, 5000, 2000, 3)
        patch = patch.convert('RGB')
        patch, highlighted_image, result_image_pil = remove_artifacts(patch)
        plot_artifact_removal(patch, highlighted_image, result_image_pil)


    Parameters:
    patch (PIL Image): The input pathology image patch.

    Returns:
    tuple: The original patch, highlighted artifacts image, and the final image with ROIs kept and non-ROIs made white.
    The patch: 'result_image_pil' is the patch without the artifacts.
    """
    # Convert PIL image to OpenCV format
    original_image = cv2.cvtColor(np.array(patch), cv2.COLOR_RGB2BGR)

    # Define the range for pink, blue, and purple colors in HSV
    lower_bounds = [np.array([120, 20, 70]), np.array([240, 20, 70]), np.array([160, 20, 70])]
    upper_bounds = [np.array([180, 255, 255]), np.array([270, 255, 255]), np.array([300, 255, 255])]

    # Remove the specified colors
    processed_image, mask = remove_colors(original_image, lower_bounds, upper_bounds)

    # Highlight the artifacts
    highlighted_image = highlight_artifacts(original_image, processed_image)

    # Create a mask from the highlighted artifacts
    highlighted_image_gray = cv2.cvtColor(highlighted_image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(highlighted_image_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to keep regions of interest
    roi_mask = np.zeros_like(highlighted_image_gray)
    cv2.drawContours(roi_mask, contours, -1, (255), thickness=cv2.FILLED)

    # Dilate the mask to include some margin around the regions of interest
    """
    1.	Kernel Size Hyperparameter:
        •	The kernel size used in the dilation process is a critical hyperparameter that affects how much the regions of interest (ROIs) are expanded.
        •	A kernel is a small, square matrix (in our case, 15x15 pixels) that is used to “slide” over the image. Wherever the kernel encounters a white    pixel (part of the ROI), it turns the entire area covered by the kernel white.
        •	The current kernel size is 15x15 pixels, and we apply it twice (iterations=2). This means that each white pixel in the ROI will expand by 15 pixels in all directions, effectively growing the ROI.
    2.	Purpose of the Kernel:
        •	The kernel is used to ensure that colors which are not specifically pink, purple, or blue, but are present in the ROIs, are also represented. This is important because the ROIs might contain colors that are close to the specified ones but not exact matches.
        •	By using a larger kernel, you can increase the margin around the ROIs, ensuring that even slightly different colors in the surrounding area are included in the ROI.
    3.	Adjusting the Kernel Size:
        •	You can adjust the kernel size to change the margin around the ROIs. A larger kernel will result in a larger margin, meaning the ROIs will expand more.
        •	For example, if you change the kernel size from 15x15 to 20x20, the dilation process will expand the ROIs by 20 pixels in all directions instead of 15 pixels.
    """
    kernel = np.ones((15, 15), np.uint8)
    roi_mask_dilated = cv2.dilate(roi_mask, kernel, iterations=2)

    # Apply the mask to the original image
    result_image = original_image.copy()
    result_image[roi_mask_dilated == 0] = [255, 255, 255]  # Make non-ROI regions white

    # Convert back to PIL format for display
    result_image_pil = Image.fromarray(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))

    return patch, highlighted_image, result_image_pil


### Remove blurriness

In [16]:
def detect_blurriness_and_contrast(image, blur_threshold=70, contrast_threshold=0.35):
    # Read the image using OpenCV
    if isinstance(image, Image.Image):
        image = np.array(image)
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the Laplacian of the image and then the variance
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    # Determine if the image is blurry
    is_blurry = laplacian_var < blur_threshold
    return is_blurry


### Patch selection

In [18]:

def decide_which_function_for_background(patch, level, final_level = False):
    _, _, result_image_pil = remove_artifacts(patch)
    if not final_level and level == 4:
        return is_patch_white(result_image_pil, 0.65, 254) 
    if not final_level and level == 3:
        return is_patch_white(result_image_pil, 0.40, 254)  or detect_blurriness_and_contrast(result_image_pil)
    if final_level:
        return is_patch_white(result_image_pil, 0.15, 254)  or detect_blurriness_and_contrast(result_image_pil)
    return is_patch_white(result_image_pil, 0.8, 254) 


def do_pyramid_level(slide, image_name, width, height, final_level, current_level, base_x, base_y, search_width, search_height, patch_size, step_size, split, patches_info, ground_truth_bboxes):
    is_final_level = (final_level == current_level)
    
    if not is_final_level:
        current_step_size = step_size 
    else:

        current_step_size = step_size
    for y in range(base_y, base_y + search_height - patch_size + current_step_size, current_step_size):
        if y > base_y + search_height - patch_size:
            patch_y = base_y + search_height - patch_size
        else:
            patch_y = y
        for x in range(base_x, base_x + search_width - patch_size + current_step_size, current_step_size):    
            if x > base_x + search_width - patch_size:
                patch_x = base_x + search_width - patch_size
            else:
                patch_x = x
           
            patch = show_image_patch(slide, patch_x, patch_y, patch_size, patch_size, current_level)
            patch = patch.convert("RGB")
            
            # Check if the patch is background
            if not decide_which_function_for_background(patch, current_level, is_final_level):
                if is_final_level:
                    patch_bbox = (patch_x, patch_x + patch_size, patch_y, patch_y + patch_size)
                    label = is_patch_positive(patch_bbox, ground_truth_bboxes)
                    patches_info['label'].append(int(label))
                    patches_info['x1'].append(patch_x)
                    patches_info['y1'].append(patch_y)
                    patches_info['filename'].append(image_name)
                    patches_info['image_width'].append(width)
                    patches_info['image_height'].append(height)
                else:
                    new_base_x, new_base_y = patch_x * 2, patch_y*2 
                    patches_info = do_pyramid_level(slide, image_name, width, height, final_level, current_level-1, new_base_x, new_base_y, patch_size*2, patch_size*2, patch_size, step_size, split, patches_info, ground_truth_bboxes)
            
    if current_level == 5:
        tqdm_pbar_images.update(1)
    return patches_info
        

def do_pyramid(image_path, level=1, patch_size=224, step_size=64):

    # Open the whole slide image
    # Open the TIFF file
    slide = openslide.OpenSlide(image_path)
    width, height = slide.level_dimensions[level]
    split = is_image_train_val_annotated(image_path.split('/')[-1])
    image_name = image_path.split('/')[-1]
    if split == 'train':
        ground_truth_bboxes, old_width, old_height= get_bounding_boxes_for_specific_image(image_path.split('/')[-1])
        ground_truth_bboxes = resize_bboxes(ground_truth_bboxes, (old_width, old_height), (width, height))
    else:
        return []
   
    level_5_width, level_5_height = slide.level_dimensions[5]
    patches_info = {'label': [], 'x1':[], 'y1':[], 'filename':[], 'image_width':[], 'image_height':[]}

    return do_pyramid_level(slide, image_name, width, height, level, 5, 0, 0, level_5_width, level_5_height, patch_size, step_size, split, patches_info, ground_truth_bboxes)

def process_image(image_path,  level, patch_size, step_size):

    
    try:
        patches_info = do_pyramid(image_path, patch_size=patch_size, step_size=step_size, level=level)
        return patches_info
    except Exception as e:
        # Handle the error by returning a failure status
        print(e, image_path)
        return []
            

## Create CSV file with suitable patch coordinates
The following section of the code generates a CSV file containing the coordinates of the suitable patches for all images in the image folder.

In [17]:

import warnings
warnings.filterwarnings('ignore')

# Get list of image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))] #[:50]

# Shared DataFrame (empty initially)
columns = ['label', 'x1',  'y1',  'filename', 'image_width', 'image_height']
df = pd.DataFrame(columns=columns)

def init(pbar_images):
    global tqdm_pbar_images
    tqdm_pbar_images = pbar_images

# Create a multiprocessing pool
# Create a tqdm progress bar
with tqdm(total=len(image_files), desc='Processing images') as pbar_images:
    with Pool(processes=16, initializer=init, initargs=(pbar_images, )) as pool:  # Adjust the number of processes as needed
        # Map the function to process each image path
        results = []
        for image_path in image_files:
                results.append(pool.apply_async(process_image, (image_path, level, patch_size, step_size)))
        
        # Retrieve results from processes
        for result in results:
            result = pd.DataFrame(result.get())
            df = df.append(result, ignore_index=True)
df = df.drop_duplicates()
# Print the final DataFrame with processed results
df.to_csv(csv_file, index=False)
print(df.head())
print(df.shape)

2 224 200


Processing images:   0%|          | 3/4415 [10:11<251:17:05, 205.04s/it]

Unsupported or missing image file ./data/images/AK7tmBa937_b.tif


Processing images:   0%|          | 12/4415 [32:04<222:29:40, 181.92s/it]

Unsupported or missing image file ./data/images/o26ULhMO2U_b.tif


Processing images:   0%|          | 0/4415 [48:30<?, ?it/s]1, 205.91s/it]


  label     x1    y1          filename image_width image_height
0     0  18200  4600  jlrFhKAJs0_a.tif       20736        49472
1     0  18224  4600  jlrFhKAJs0_a.tif       20736        49472
2     0  18200  4624  jlrFhKAJs0_a.tif       20736        49472
3     0  18224  4624  jlrFhKAJs0_a.tif       20736        49472
4     0  18048  4600  jlrFhKAJs0_a.tif       20736        49472
(2169585, 6)


In [20]:
def compute_coverage(bbox_gt, bbox_patch, bbox_gt_area):
    """
    Compute the coverage of bounding box 1 by bounding box 2.
    
    Parameters:
    - box1, box2: Lists or tuples of format [x1, y1, x2, y2] where
      (x1, y1) is the top-left coordinate and (x2, y2) is the bottom-right coordinate.
    
    Returns:
    - coverage: The proportion of box 1 covered by box 2.
    """
    
    # Unpack the coordinates of both bounding boxes
    x1_min, x1_max, y1_min, y1_max = bbox_gt
    x2_min, x2_max, y2_min, y2_max = bbox_patch
    
    # Calculate the (x, y)-coordinates of the intersection rectangle
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    
    # Compute the area of the intersection rectangle
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    inter_area = inter_width * inter_height
    
    # Compute the coverage
    coverage = inter_area / bbox_gt_area if bbox_gt_area != 0 else 0
    
    return coverage

def are_extracted_patches_valid(patches_path, annotation_path, threshold=0.75):
    patches_df = pd.read_csv(patches_path)
    ann_df = pd.read_csv(annotation_path)
    print(ann_df.head())

    print(patches_df.head())
    
    missing = []
    count = 0
    for _, image_row in tqdm(ann_df.iterrows()):
        bbox_gt = image_row["x1"], image_row["x2"], image_row["y1"], image_row["y2"]

        coverage = 0
        filename = image_row['filename']
        old_width, old_height = image_row['max_x'], image_row['max_y']

        image_patches = patches_df[patches_df['filename'] == filename]
        
        image_patches =image_patches.reset_index()
        if image_patches.shape[0] == 0:
            missing.append(filename)
        else:
            width, height= image_patches.iloc[0]['image_width'], image_patches.iloc[0]['image_height']

            bbox_gt = resize_bboxes([bbox_gt], (old_width, old_height), (width, height))[0]
            bbox_gt_area = (bbox_gt[1] - bbox_gt[0]) * (bbox_gt[3]- bbox_gt[2])
            for _, patch_row in image_patches.iterrows():
                bbox_patch = patch_row["x1"], patch_row["x1"]+224, patch_row["y1"], patch_row["y1"]+224
                patch_coverage = compute_coverage(bbox_gt, bbox_patch, bbox_gt_area)
                coverage += patch_coverage
            if  coverage < threshold:
                count += 1
    return count, missing


annotation_path_train = 'clean_train.csv'

count, missing = are_extracted_patches_valid(patches_path, annotation_path_train, threshold=0.9)
print(count, len(missing))

   Unnamed: 0  index          filename     x1     x2     y1     y2  max_x  \
0           0      0  bGaslniO4a_a.tif  29348  30108  28404  29675  82944   
1           1      1  bGaslniO4a_a.tif  11735  12379  70274  71195  82944   
2           2      2  2qj5MlLLBT_a.tif  11185  12276  11571  12671  82944   
3           3      3  2qj5MlLLBT_a.tif  14380  15583  11252  12434  82944   
4           4      4  2qj5MlLLBT_a.tif  12162  13834  71136  72440  82944   

    max_y  
0  197632  
1  197632  
2  196608  
3  196608  
4  196608  
   label     x1    y1          filename  image_width  image_height
0      0  18200  4600  jlrFhKAJs0_a.tif        20736         49472
1      0  18224  4600  jlrFhKAJs0_a.tif        20736         49472
2      0  18200  4624  jlrFhKAJs0_a.tif        20736         49472
3      0  18224  4624  jlrFhKAJs0_a.tif        20736         49472
4      0  18048  4600  jlrFhKAJs0_a.tif        20736         49472


910it [04:04,  3.73it/s]

248 0


The following code reads the CSV file containing the patch coordinates, splits the data into training and validation sets based on IDs from new_val_ids.txt, balances the training set by oversampling patches that are parts of a lesion, and saves the processed training and validation sets to separate CSV files.

In [34]:
seed = 42

patches_df = pd.read_csv(patches_path)
print(patches_df.shape)

train_csv_file = os.path.join(csv_dir, 'train.csv')
val_csv_file = os.path.join(csv_dir, 'corrected_val.csv')

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

image_ids = patches_df['filename'].unique()
print(len(image_ids))
val_ids = list(train_test_split(image_ids, test_size=0.03, shuffle=True, random_state=42)[1])
print(val_ids)


print(len(val_ids))
with open('new_val_ids.txt', mode='w') as file:
    for number in val_ids:
        file.write(f"{number}\n")

train_df = patches_df[patches_df['filename'].isin(val_ids) == False].reset_index(drop=True)
val_df = patches_df[patches_df['filename'].isin(val_ids)].reset_index(drop=True)

print(train_df.shape)
print(val_df.shape)


train_label_1_df = train_df[train_df['label'] == 1]
n = len(train_label_1_df)
print(n)

train_label_0_df = train_df[train_df['label'] == 0].sample(n=2*n, random_state=seed)

all_train_df_sampled = pd.concat([train_label_0_df, train_label_1_df])

print(all_train_df_sampled.shape)

all_train_df_sampled.to_csv(train_csv_file, index=False)
val_df.to_csv(val_csv_file, index=False)


(2169585, 6)
247
['kiAdmoDDMM_a.tif', 'Mn1A3CnXrc_b.tif', '0w9NQUKyFU_b.tif', 'fgh7blkYnD_b.tif', 'StKznQnYkK_a.tif', '4y4EEMpL4L_a.tif', '51nZ7GBlix_b.tif', 'lYkGPxzccV_a.tif']
8
(2106665, 6)
(62920, 6)
4898
(14694, 6)


In [38]:
def extract_patches_from_images(data_path, csv_folder, zip_name, patch_size=224, level=0):
    # Open the whole slide image
    train_df = pd.read_csv(os.path.join(csv_folder, 'train.csv'))
    val_df = pd.read_csv(os.path.join(csv_folder, 'corrected_val.csv'))
    all_df = pd.concat([train_df, val_df])
    print(all_df.shape)
    grouped = all_df.groupby('filename')
    dfs_by_imagename = [group for _, group in grouped]
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for df in tqdm(dfs_by_imagename):
            df = df.reset_index()
            file_name =  df.loc[0, 'filename']
            slide_path = os.path.join(data_path, file_name)
            slide = openslide.OpenSlide(slide_path)
            
            width, height = slide.dimensions

            for _, patch_row in  df.iterrows():
                # Get the patch
                x, y = patch_row['x1'], patch_row['y1']
                base_x, base_y, _, _ = scaling_images(slide, x, y, patch_size, patch_size, level)
                patch = slide.read_region((base_x, base_y), level, (patch_size, patch_size)).convert('RGB')
                _, _, patch = remove_artifacts(patch)
                try:
                    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                        temp_path = temp_file.name
                        patch.save(temp_path, format='PNG')
                        zipf.write(temp_path, os.path.basename(f"{file_name.split('.')[0]}_{x}_{y}_{patch_size}.png"))
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")


extract_patches_from_images(folder_path, csv_dir, zip_name, patch_size, level=level)


(77614, 6)


100%|██████████| 247/247 [17:57<00:00,  4.36s/it]  
